In [ ]:


from instrument import *
from tkinter import *
from consts import *
import sounddevice as sd
import numpy as np
import mido
import time 


colors = ["blue", "red", "yellow", "pink"]


class MidiSequencerTkPLUS:
    # análogo a lo anterior
    def __init__(self,tk,instrument=None, midis=None):
        self.instruments = instrument

        #si se pasa un vector vacio se settea a un instrumento
        if(len(instrument) == 0):
            self.instruments.append(Instrument(tk,amp=0.2,ratio=3,beta=0.6))

        self.tick = 1
        self.state = 'off'

        c = 0
        j = 0

        self.files = []
        self.text = []
        self.playBut = []
        self.stopBut = []
        # trato instrument com oun vector d e instrumentos
        for i in instrument:
            self.instruments[j] = i

            frame = LabelFrame(tk, text="Midi Sequencer", bg="#908060")
            frame.pack(side=TOP)
            frameFile = Frame(frame, highlightbackground=colors[c], highlightthickness=6)
            
            c += 1
            if(c == len(colors)): 
                c = 0
                
            frameFile.pack(side=TOP)
            Label(frameFile,text='Archivo MIDI: ').pack(side=LEFT)

            if(j < len(midis)):
                self.files.append(Entry(frameFile)) #.pack(side=RIGHT)
                self.files[j].insert(14,midis[j])
                self.files[j].pack(side=LEFT)
             
                
            self.transport = 0
            
            self.text.append(Text(frame,height=6,width=23))
            self.text[j].pack(side=RIGHT)
            self.playBut.append(Button(frame,text="Play", command=self.caller(j, self.play)))
            self.playBut[j].pack(side=TOP)
            self.stopBut.append(Button(frame,text="Stop", command=self.caller(j, self.stop)))
            self.stopBut[j].pack(side=BOTTOM)
            
            j += 1
            
        print(self.files)
        
    # obtención de la secuencia midi (noteOn/Off) con tiempos relativos al inicio
    def getSeq(self,midiEvents, i=0):
        seq = []
        accTime = 0
        for m in midiEvents:
            accTime += m.time
            if m.type=='note_on':
                if m.velocity==0: seq.append((accTime,'noteOff',m.note+self.transport,i))
                else: seq.append((accTime,'noteOn',m.note+self.transport, i))    
            elif m.type=='note_off':
                seq.append((accTime,'noteOff',m.note+self.transport,i))
        return seq

  

    def caller(self, i, cb):
        cb(i)


    def play(self, i=0):
        events = mido.MidiFile(self.files[i].get())
        seq = self.getSeq(events, i)
        #print(seq)
        self.state = 'on'
        self.playLoop(seq, i)

    def playLoop(self,seq,item=0,accTime=0, i=0):  

        print("llega") 
        if item>=len(seq) or self.state =='off':
            return
        # ahora tenemos que procesar todos los ítems cuyo tiempo supere el crono accTime    
        while item<len(seq) and accTime>=seq[item][0]:
            (_,msg,midiNote,_chan) = seq[item]  # (time,'noteOff',midNote,channel)
            self.text[i].insert('6.0',  f'{msg} {midiNote}\n')  #CAMBIAR ESTO
            if msg=='noteOn':  
                self.instruments[i].noteOn(midiNote)                   
            else: # msg noteOff    
                self.instruments[i].noteOff(midiNote)                   
            item += 1 # y avanzmos ítem


        # avanzammos crono 
        accTime += self.tick/1000

        self.text[i].after(self.tick,lambda: self.playLoop(seq,item,accTime)) 

         
    def stop(self, i=0):
        self.instruments[i].stop()
        self.state = 'off'  


In [17]:
%load_ext autoreload
%autoreload 2

from midiSequencerTk import *
import os    
from instrument import *


def test():
    def callback(outdata, frames, time, status):    
        if status: print(status)    
        #print(inputs)
        s = np.sum([i.next() for i in inputs],axis=0)
        s = np.float32(s)
        outdata[:] = s.reshape(-1, 1)

    os.system('xset r off')
    tk = Tk()

    instrumentos = []
    instrumentos.append(Instrument(tk, "Holaaa", 0.3, 4, 0.5))
    instrumentos.append(Instrument(tk, "Adiosss", 0.1, 3, 0.7))

    print(instrumentos)
    #instrumentos.append(Instrument(tk, "Huh"))

    midis = ["bumblebee.mid", "pirates.mid"]

    seq = MidiSequencerTkPLUS(tk,instrumentos, midis)
    #print(seq.seq)
    inputs = instrumentos

    stream = sd.OutputStream(samplerate=SRATE, channels=1, blocksize=CHUNK, callback=callback)
    stream.start()

    tk.mainloop()

    stream.close()
    os.system('xset r on')

test()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[<instrument.Instrument object at 0x0000022540A4CD90>, <instrument.Instrument object at 0x000002255ECF2090>]
llega


TypeError: 'NoneType' object is not callable